In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/abpal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
documents = docs
len(documents)

18846

In [3]:
documents[0]

"\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [4]:
def preprocess_data(documents):
 stop_words = stopwords.words("english")
 
 # Tokenize and remove stopwords
 texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in documents]
 
 return texts

processed_texts = preprocess_data(documents)


In [5]:
# Create Dictionary
id2word = corpora.Dictionary(processed_texts)
# Create Corpus
texts = processed_texts
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [6]:
# Set number of topics
num_topics = 100
# Build LDA model
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42, passes=10, alpha="auto", per_word_topics=True)

In [19]:
# Print the keywords for each topic
pprint(lda_model.print_topics())

[(44,
  '0.819*"ax" + 0.061*"max" + 0.008*"pl" + 0.007*"ei" + 0.006*"giz" + '
  '0.006*"tm" + 0.005*"di" + 0.005*"wm" + 0.004*"ey" + 0.003*"tq"'),
 (65,
  '0.067*"northern" + 0.038*"ne" + 0.035*"muscle" + 0.030*"triple" + '
  '0.027*"est" + 0.026*"je" + 0.025*"qu" + 0.022*"programme" + 0.022*"ce" + '
  '0.021*"le"'),
 (35,
  '0.021*"mk" + 0.018*"ah" + 0.015*"pl" + 0.014*"di" + 0.013*"mq" + 0.013*"sl" '
  '+ 0.011*"mn" + 0.010*"tm" + 0.009*"wm" + 0.009*"m_"'),
 (42,
  '0.085*"echo" + 0.082*"lc" + 0.039*"cg" + 0.033*"defined" + 0.033*"define" + '
  '0.031*"ivf" + 0.028*"tex" + 0.024*"ar" + 0.021*"dd" + 0.019*"ted"'),
 (78,
  '0.047*"turkish" + 0.046*"greek" + 0.037*"turkey" + 0.027*"greece" + '
  '0.022*"turks" + 0.017*"ed" + 0.016*"ottoman" + 0.015*"population" + '
  '0.015*"genocide" + 0.014*"istanbul"'),
 (23,
  '0.107*"theory" + 0.035*"comprehensive" + 0.027*"ec" + 0.024*"desk" + '
  '0.024*"optical" + 0.020*"evolution" + 0.020*"corporate" + '
  '0.018*"accurately" + 0.014*"tutorial"

In [20]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_texts, dictionary=id2word, coherence="c_v")
coherence_lda = coherence_model_lda.get_coherence()
print("Coherence Score: ", coherence_lda)

Coherence Score:  0.5776705610954899


In [14]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model,corpus=corpus,dictionary=id2word)

In [15]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
43    -0.331271 -0.111582       1        1  8.938990
69    -0.340633 -0.082017       2        1  8.368654
44    -0.003230 -0.014687       3        1  4.376253
8     -0.182374 -0.157949       4        1  2.491302
96    -0.205447  0.164385       5        1  2.305261
...         ...       ...     ...      ...       ...
42     0.161662  0.014515      96        1  0.149248
36     0.172042  0.003321      97        1  0.148274
10     0.176319  0.000971      98        1  0.138346
23     0.176820  0.005354      99        1  0.134464
65     0.202589  0.006012     100        1  0.093086

[100 rows x 5 columns], topic_info=          Term          Freq         Total  Category  logprob  loglift
23830       ax  68818.000000  68818.000000   Default  30.0000  30.0000
791      would  10921.000000  10921.000000   Default  29.0000  29.0000
323        max   5153.000000   5153.000000   Default  28.0000  28.0000
1732       edu   4032.000000   4032.000000   Default  27.0000  27.0000
3127       god   4182.000000   4182.000000   Default  26.0000  26.0000
...        ...           ...           ...       ...      ...      ...
8086        il     36.738366    121.120902  Topic100  -3.8845   5.7864
4365        hc     23.567959     81.504314  Topic100  -4.3285   5.7386
6875   collect     15.337266     62.628044  Topic100  -4.7581   5.5725
5309        ku     20.004258    166.789792  Topic100  -4.4924   4.8586
5857       bar     13.304816    127.326245  Topic100  -4.9002   4.7208

[5668 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
3367      26  0.104214       aa
3367      63  0.892333       aa
2542      97  0.980465      aaa
291       28  0.008624      abc
291       33  0.983129      abc
...      ...       ...      ...
17413     63  0.988010       zo
8592      48  0.952142  zooming
25154      3  0.961722     zrlk
25161     26  0.988652       zt
9740      98  0.988487       zx

[15533 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[44, 70, 45, 9, 97, 15, 46, 74, 55, 88, 65, 61, 38, 23, 87, 22, 4, 48, 80, 92, 20, 8, 3, 34, 1, 36, 19, 10, 2, 32, 73, 99, 81, 91, 89, 53, 57, 17, 50, 16, 67, 93, 71, 54, 79, 72, 42, 100, 56, 63, 90, 58, 62, 21, 78, 94, 31, 33, 28, 52, 75, 85, 84, 60, 27, 96, 13, 30, 25, 39, 77, 6, 14, 68, 29, 18, 35, 5, 76, 64, 95, 86, 98, 12, 83, 69, 7, 82, 26, 59, 49, 40, 47, 41, 51, 43, 37, 11, 24, 66])

In [11]:
x= lda_model.show_topics(num_topics=100, num_words=10,formatted=False)
# x= lda_model.show_topics()
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Topics and Words
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

#Below Code Prints Only Words 
for topic,words in topics_words:
    print(" ".join(words))

0::['god', 'christian', 'religion', 'believe', 'christians', 'belief', 'religious', 'atheism', 'people', 'bible']
1::['book', 'books', 'history', 'press', 'first', 'university', 'professor', 'general', 'published', 'page']
2::['space', 'nasa', 'earth', 'launch', 'orbit', 'moon', 'solar', 'spacecraft', 'satellite', 'planet']
3::['went', 'came', 'back', 'said', 'one', 'home', 'took', 'told', 'saw', 'started']
4::['printer', 'hp', 'paper', 'print', 'laser', 'fonts', 'printing', 'printers', 'ink', 'bob']
5::['procedure', 'catholics', 'latin', 'mass', 'doug', 'periods', 'sales', 'scan', 'wright', 'masses']
6::['xview', 'openwindows', 'japanese', 'lds', 'encourage', 'patch', 'pointer', 'shared', 'bugs', 'languages']
7::['anyone', 'thanks', 'would', 'know', 'please', 'help', 'like', 'could', 'looking', 'hi']
8::['government', 'war', 'rights', 'people', 'world', 'countries', 'human', 'power', 'political', 'country']
9::['edu', 'com', 'cs', 'apr', 'article', 'ca', 'david', 'subject', 'gov', 'ma

In [13]:
x=lda_model.show_topics()

twords={}
for topic,word in x:
    twords[topic]=re.sub('[^A-Za-z ]+', '', word)
print(twords)

{44: 'ax  max  pl  ei  giz  tm  di  wm  ey  tq', 65: 'northern  ne  muscle  triple  est  je  qu  programme  ce  le', 35: 'mk  ah  pl  di  mq  sl  mn  tm  wm  m', 42: 'echo  lc  cg  defined  define  ivf  tex  ar  dd  ted', 78: 'turkish  greek  turkey  greece  turks  ed  ottoman  population  genocide  istanbul', 14: 'would  money  make  much  cost  pay  use  one  could  private', 7: 'anyone  thanks  would  know  please  help  like  could  looking  hi', 54: 'get  one  like  good  got  back  bike  well  new  right', 69: 'one  would  people  think  even  like  see  say  may  way', 43: 'think  know  people  get  would  like  going  want  time  go'}


In [25]:
def run_lda(docs: list,repeats : int,savepath : str, topn : int) -> None:
    """Run LDA on the documents, and save the avg Coherence rating in the given path

    Args:
        docs (list): _description_
        repeats (int): _description_
        savepath (str): _description_
    """

    coherences = []
    for _ in range(repeats):
        processed_texts = preprocess_data(docs)

        # Create Dictionary
        id2word = corpora.Dictionary(processed_texts)
        # Create Corpus
        texts = processed_texts
        # Term Document Frequency
        corpus = [id2word.doc2bow(text) for text in texts]
    
        # Set number of topics
        num_topics = 100
        # Build LDA model
        lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42, passes=10, alpha="auto", per_word_topics=True)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_texts, dictionary=id2word, coherence="c_v", topn = topn) # types {'u_mass', 'c_v', 'c_uci', 'c_npmi'}
        coherence_lda = coherence_model_lda.get_coherence()
        coherences.append(coherence_lda)

    with open(savepath,"w") as sp:
        sp.write("".join(str(coherences)))
        sp.write(f"Mean : {np.mean(coherences)} STD : {np.std(coherences)}")
        

In [26]:
run_lda(documents,3,"final_result/LDA/LDA_Result_20ng.txt",topn=5)

In [27]:
def read_data(path):
    df = pd.read_csv(path)
    return df['text'].tolist()

docs = read_data("data/nyt2020.csv")
run_lda(docs,3,"final_result/LDA/LDA_Result_nyt.txt",5)

In [28]:
docs = read_data("data/wiki_en_10000.csv")
run_lda(docs,3,"final_result/LDA/LDA_Result_wiki.txt",5)

# Octis for NYT, Wiki

In [16]:
def save_topics_lda(docs):
    processed_texts = preprocess_data(docs)

    # Create Dictionary
    id2word = corpora.Dictionary(processed_texts)
    # Create Corpus
    texts = processed_texts
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # Set number of topics
    num_topics = 100
    # Build LDA model
    lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=42, passes=10, alpha="auto", per_word_topics=True)
    x= lda_model.show_topics(num_topics=100, num_words=10,formatted=False)
    # x= lda_model.show_topics()
    topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

    #Below Code Prints Topics and Words
    for topic,words in topics_words:
        print(str(topic)+ "::"+ str(words))
    print()

    #Below Code Prints Only Words 
    for topic,words in topics_words:
        print(" ".join(words))


In [22]:
nyt = list(pd.read_csv("data/nyt2020.csv")["text"])
wiki = list(pd.read_csv("data/wiki_en_10000.csv")["text"])

In [23]:
save_topics_lda(nyt)

0::['new', 'york', 'city', 'week', 'times', 'change', 'climate', 'baseball', 'homes', 'tales']
1::['lives', 'book', 'inside', 'books', 'read', 'pay', 'talk', 'center', 'listen', 'ask']
2::['party', 'voters', 'republican', 'despite', 'short', 'term', 'biggest', 'hold', 'democratic', 'pressure']
3::['came', 'harris', 'battle', 'six', 'holidays', 'kamala', 'wisconsin', 'field', 'threats', 'water']
4::['end', 'care', 'wave', 'fall', 'likely', 'access', 'generation', 'room', 'host', 'cancer']
5::['control', 'russia', 'south', 'military', 'victory', 'perfect', 'security', 'africa', 'amy', 'wide']
6::['future', 'service', 'variety', 'someone', 'forget', 'races', 'income', 'train', 'emily', 'friday']
7::['trump', 'president', 'election', 'republicans', 'administration', 'said', 'federal', 'democrats', 'vote', 'voting']
8::['british', 'street', 'park', 'wall', 'defense', 'asking', 'drawing', 'traditional', 'shootings', 'inspire']
9::['among', 'let', 'key', 'fear', 'remain', 'allies', 'many', 't

In [24]:
save_topics_lda(wiki)

0::['published', 'book', 'writers', 'books', 'russian', 'american', 'press', 'century', 'women', 'people']
1::['school', 'university', 'college', 'high', 'students', 'schools', 'new', 'state', 'building', 'education']
2::['river', 'county', 'lake', 'state', 'rivers', 'states', 'illinois', 'wisconsin', 'arkansas', 'united']
3::['acc', 'jeeves', 'beryl', 'antioxidant', 'difranco', 'ayckbourn', 'tae', 'jarvis', 'patel', 'antioxidants']
4::['color', 'strip', 'comic', 'red', 'paper', 'comics', 'artists', 'strips', 'colors', 'capp']
5::['germany', 'german', 'european', 'europe', 'republic', 'croatia', 'states', 'war', 'united', 'czech']
6::['apollo', 'space', 'earth', 'star', 'mission', 'moon', 'lunar', 'crew', 'nasa', 'stars']
7::['economic', 'world', 'million', 'country', 'economy', 'trade', 'largest', 'bank', 'industry', 'government']
8::['cay', 'bulgaria', 'bulgarian', 'azerbaijan', 'union', 'armenia', 'trade', 'armenian', 'russian', 'columbus']
9::['augustine', 'td', 'ax', 'fianna', 'fá